In [ ]:
import torch
import torchvision
from torchvision import datasets,transforms
import os
import torch.nn as nn
import torch.optim as optim
import pandas as pd

### Data preparation

In [ ]:
dir = '../data/combined_dataset/'

params = { 'batch_size':16,
           'shuffle':True,
           'num_workers':4 }

# train dataset
transform = transforms.Compose([transforms.Resize(256),
                                transforms.RandomResizedCrop(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
train_dataset = datasets.ImageFolder(os.path.join(dir, 'train'),transform = transform )

# validation dataset
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
dir = '../data/dataset_split'
val_dataset = datasets.ImageFolder(os.path.join(dir, 'val'),transform = transform )


train_dataloader = torch.utils.data.DataLoader(train_dataset, **params)
val_dataloader = torch.utils.data.DataLoader(val_dataset, **params)

class_names = train_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
print('Train dataset = {}\n'.format(len(train_dataset)),'Val dataset = {}'.format(len(val_dataset)))
print('Classes = {}'.format(class_names))

Train dataset = 8832
 Val dataset = 719
Classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


### Training Loop for Model Fine-Tuning with Epoch-wise Performance Tracking and Save to CSV

In [ ]:
def train(model, loss_fn, optimizer, num_epochs=25, save_path='trained_combined_augmentations_version_01_val_densenet.csv'):
    best_acc = 0
    results = []

    for epoch in range(num_epochs):
        
        print('Epoch {}'.format(epoch+1))
        
        # Train dataset
        model.train()
        train_loss = 0.0
        train_correct = 0
        size = len(train_dataset)
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            train_correct += torch.sum(preds == labels.data)
            
        train_loss = train_loss / size
        train_acc = train_correct.double() / size
            
        print('Training Loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc))
        
        # Validation dataset
        model.eval()
        val_loss = 0.0
        val_correct = 0
        size = len(val_dataset)
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = loss_fn(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_correct += torch.sum(preds == labels.data) 

        val_loss = val_loss / size
        val_acc = val_correct.double() / size    

        print('Validation Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))    
            
        if val_acc > best_acc:
            best_acc = val_acc

        results.append([epoch + 1, [train_loss], [train_acc.item()], [val_loss], [val_acc.item()]])
    
    df = pd.DataFrame(results, columns=['Epoch', 'Train Loss', 'Train Accuracy', 'Validation Loss', 'Validation Accuracy'])
    df.to_csv(save_path, index=False)
    
    print('Best Validation Accuracy: {:.4f}'.format(best_acc))
    print(f'Training results saved to {save_path}')
    
    return model

### Fine-Tuning DenseNet-121

In [ ]:
net = torchvision.models.densenet121(pretrained=True)
for param in net.parameters():
    param.requires_grad = False

ft = net.classifier.in_features
net.classifier = nn.Linear(ft, 6)

net = net.to(device)

loss = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.classifier.parameters(), lr=0.0001, momentum=0.9) 

### Train the model

In [ ]:
net = train(net,loss,optimizer,num_epochs=100)

Epoch 1
Training Loss: 1.6024 Acc: 0.3682
Validation Loss: 1.2911 Acc: 0.5925
Epoch 2
Training Loss: 1.3369 Acc: 0.5302
Validation Loss: 1.0846 Acc: 0.6676
Epoch 3
Training Loss: 1.2231 Acc: 0.5731
Validation Loss: 0.9833 Acc: 0.6982
Epoch 4
Training Loss: 1.1587 Acc: 0.5931
Validation Loss: 0.9110 Acc: 0.7093
Epoch 5
Training Loss: 1.1045 Acc: 0.6120
Validation Loss: 0.8681 Acc: 0.7149
Epoch 6
Training Loss: 1.0655 Acc: 0.6192
Validation Loss: 0.8421 Acc: 0.7330
Epoch 7
Training Loss: 1.0434 Acc: 0.6298
Validation Loss: 0.8196 Acc: 0.7232
Epoch 8
Training Loss: 1.0272 Acc: 0.6291
Validation Loss: 0.8079 Acc: 0.7302
Epoch 9
Training Loss: 1.0089 Acc: 0.6377
Validation Loss: 0.7854 Acc: 0.7371
Epoch 10
Training Loss: 0.9963 Acc: 0.6409
Validation Loss: 0.7624 Acc: 0.7497
Epoch 11
Training Loss: 0.9883 Acc: 0.6447
Validation Loss: 0.7448 Acc: 0.7552
Epoch 12
Training Loss: 0.9648 Acc: 0.6574
Validation Loss: 0.7468 Acc: 0.7469
Epoch 13
Training Loss: 0.9617 Acc: 0.6504
Validation Loss: 0

### Saving the parameters of the desnet model after training

In [ ]:
torch.save(net.state_dict(), "trained_combined_modell_val_densenet.pth")  # Save model weights
print("Model saved as trained_densenet.pth")


Model saved as trained_densenet.pth
